In [13]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
import seaborn as sns; sns.set(style="ticks", color_codes="pastel")
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import tree
import time
from sklearn import metrics
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss


def stdscaler(df):
    scaler = StandardScaler()
    scaler.fit(df)
    scX_df = scaler.transform(df)
    df=pd.DataFrame(scX_df, index =df.index,columns =df.columns  )
    return df


import time

samplechoice = "NearMiss"
"""
if samplechoice == "SMOTE":
    smt = SMOTE()
    train_data , train_y = smt.fit_sample(train_data , train_y)
elif samplechoice == "NearMiss":
    nr = NearMiss()
    train_data , train_y = nr.fit_sample(train_data , train_y)

    
"""


def kfoldtest(kfoldn,model,Xtrain_df,ytrain_df):
    kf=StratifiedKFold(n_splits=kfoldn)
    accu=[]
    auc=[]
    f1 =[]
    aab =[]
    dcd = []
    tim = []
    kf.get_n_splits(Xtrain_df,ytrain_df)
    
    for train_index, test_index in kf.split(Xtrain_df,ytrain_df):
        #print("TRAIN:", len(train_index), "TEST:", len(test_index))
        t1 =time.time()
        X_train, X_test = Xtrain_df.iloc[train_index], Xtrain_df.iloc[test_index]
        y_train, y_test = ytrain_df.iloc[train_index], ytrain_df.iloc[test_index]
        
        
        if samplechoice == "SMOTE":
            smt = SMOTE()
            X_train,  y_train = smt.fit_sample(X_train , y_train)
        elif samplechoice == "NearMiss":
            nr = NearMiss()
            X_train,  y_train = nr.fit_sample(X_train , y_train)
            
    
        model.fit(X_train,y_train)
        try:
            accu.append(model.score(X_test,y_test))
        except:
            print("This model have no build-in score.")
            
        auc.append(roc_auc_score(y_test,model.predict(X_test)))
        f1.append(f1_score(y_test, model.predict(X_test).astype(int),average='binary'))
        tim.append(time.time()-t1)
        cm1=confusion_matrix(test_y, clf.predict(test_data),labels = [1,0])
        sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
        aab.append(sensitivity1)

        specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
        dcd.append(specificity1)
    return accu,auc,f1,aab,dcd,tim

sns.set_style("whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})
file_name = 'UCI_Credit_Card 2.csv'
data =pd.read_csv(file_name,index_col = 0)
print(data.columns)


Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month'],
      dtype='object')


In [14]:
v1 = data[data["default.payment.next.month"]==1].copy()
v0 = data[data["default.payment.next.month"]==0].copy()
v1.rename(columns={"default.payment.next.month": "default"},inplace = True)
v0.rename(columns={"default.payment.next.month": "default"},inplace = True)
y1 = v1["default"].copy()
x1 = v1.copy()
del x1["default"]
y0 = v0["default"].copy()
x0 = v0.copy()
del x0["default"]

tr1,te1,try1,tey1 = train_test_split(
    x1, y1, test_size=0.20, random_state=42)
tr2,te2,try2,tey2 = train_test_split(
    x0, y0, test_size=0.20, random_state=42)
####
#tr2=tr2.sample(len(tr1.index))

#try2 = try2.loc[tr2.index].copy()
####


train_data=pd.concat([tr1,tr2],axis=0)

train_y = pd.concat([try1,try2],axis=0)
test_data=pd.concat([te1,te2],axis=0)

test_y = pd.concat([tey1,tey2],axis=0)


train_data = stdscaler(train_data )
test_data = stdscaler(test_data )


In [15]:

"""pca =PCA(n_components=.99).fit(train_data)
train_data = pd.DataFrame(pca.transform(train_data),index = train_data.index)   

test_data= pd.DataFrame(pca.transform(test_data),index = test_data.index)  
"""

'pca =PCA(n_components=.99).fit(train_data)\ntrain_data = pd.DataFrame(pca.transform(train_data),index = train_data.index)   \n\ntest_data= pd.DataFrame(pca.transform(test_data),index = test_data.index)  \n'

In [16]:
result=[]

clf=LogisticRegression(tol=0.01, C=0.1,solver='lbfgs', max_iter=10000,)
accu,auc,f1,aab,dcd,tim =kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["LogisticRegression",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))

1
1
1
1
1


In [5]:
clf = tree.DecisionTreeClassifier(max_depth=10,class_weight="balanced",random_state=0)
accu,auc,f1,aab,dcd,tim =kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["DecisionTree",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))

In [6]:
clf = GaussianNB(priors=None, var_smoothing=1e-09)
accu,auc,f1,aab,dcd,tim =kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["Naive bayes",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))

In [7]:
clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

accu,auc,f1,aab,dcd,tim=kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["RandomForest",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



In [8]:
clf=MLPClassifier(activation='tanh', alpha=0.05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=3000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='sgd', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

accu,auc,f1,aab,dcd,tim =kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["MLP",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))



In [9]:
clf = AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features='auto',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         
                                                         random_state=0,
                                                         splitter='random'),
                   learning_rate=1.0, n_estimators=20, random_state=None)

accu,auc,f1,aab,dcd,tim =kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["AdaboostDecisionTree",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))

In [10]:
clf= GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.15, loss='deviance', max_depth=8,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           min_weight_fraction_leaf=0.0, n_estimators=10,
                           n_iter_no_change=None,
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
accu,auc,f1,aab,dcd ,tim=kfoldtest(5,clf,train_data,train_y)
result.append(pd.DataFrame([["GradientBoosting",np.average(accu),np.average(auc),
                       np.average(f1),np.average(aab),np.average(dcd),np.average(tim)]],
                     columns=["model names",'CVaccuracy','CVaucroc','CVF1',"CVsensitivity",'CVspecificity','CVtime(s)']))


In [11]:
result= pd.concat(result,axis=0)
result.to_csv('OGmodel_{}sampling.csv'.format(str(samplechoice)))

In [12]:
result


,model names,CVaccuracy,CVaucroc,CVF1,CVsensitivity,CVspecificity,CVtime(s)
0,LogisticRegression,0.502896,0.559931,0.370781,0.652711,0.437706,1.473500
0,DecisionTree,0.559567,0.605551,0.409005,0.596837,0.599401,1.513342
0,Naive bayes,0.534814,0.473588,0.256954,0.339006,0.585020,1.458550
0,RandomForest,0.530523,0.606467,0.411733,0.687651,0.515643,7.154964
0,MLP,0.467228,0.578580,0.392527,0.762801,0.381982,24.397267
0,AdaboostDecisionTree,0.484520,0.579897,0.391921,0.715361,0.424395,1.637433
0,GradientBoosting,0.605360,0.632253,0.433233,0.609639,0.617804,1.517049
